In [14]:
import json
import requests

from enum import Enum
from collections import Counter

class Mark(Enum):
    GREEN = 0
    ORANGE = 5
    RED = 10

In [15]:
with open("response.json", "r", encoding="utf-8") as file:
        json_file = json.loads(file.read())

In [16]:
def find_meta_value(data, meta):
  """
  Находит первый объект с заданным значением ключа "meta".

  Args:
    data: Данные в формате словаря или списка.
    meta: Искомое значение ключа "meta".

  Returns:
    Словарь с найденным объектом или None, если объект не найден.
  """
  if isinstance(data, dict):
    if 'meta' in data and data['meta'] == meta:
      return data
    for value in data.values():
      result = find_meta_value(value, meta)
      if result:
        return result
  elif isinstance(data, list):
    for item in data:
      result = find_meta_value(item, meta)
      if result:
        return result
  return None

In [17]:
def find_name_value(data, name):
    """
     Function to find an object with a given key "name" and its value.

     Args:
     data: Data in dictionary format.
     name: The searched value of the key "name".

     Returns:
     A dictionary with the found object or None if the object is not found.
    """

    if 'name' in data and data['name'] == name:
        return data

    for item in data.values():
        if isinstance(item, dict):
            result = find_name_value(item, name)
            if result:
                return result
        elif isinstance(item, list):
            for sub_item in item:
                result = find_name_value(sub_item, name)
                if result:
                    return result

    return None

In [18]:
def find_nested_element(data, key1, value1, key2, value2):
    def find_first_pair(data, key1, value1):
        if isinstance(data, dict):
            if key1 in data and data[key1] == value1:
                return data
            for key, value in data.items():
                if isinstance(value, dict):
                    result = find_first_pair(value, key1, value1)
                    if result is not None:
                        return result
                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            result = find_first_pair(item, key1, value1)
                            if result is not None:
                                return result
        elif isinstance(data, list):
            for item in data:
                result = find_first_pair(item, key1, value1)
                if result is not None:
                    return result
        return None

    first_pair_dict = find_first_pair(data, key1, value1)

    if first_pair_dict:
        return find_first_pair(first_pair_dict, key2, value2)

    return None

In [19]:
def monogas_sim(json1,json2):
    gas_class1 = find_meta_value(json1,"Класс газов")
    gas_class2 = find_meta_value(json2,"Класс газов")

    
    if json1 is None and  json2 is not None:
        return Mark.RED 

    elif json1 is None and  json2 is  None:
        return Mark.GREEN
    
    #1.у газов разные классы
    if gas_class1["name"] != gas_class2["name"]:
        return Mark.RED
    else: 
        gas_name1 = find_meta_value(gas_class1,'Газ')
        gas_name2 = find_meta_value(gas_class2,'Газ')

        #2.у газов не одинаковые имена
        if gas_name1["name"] != gas_name2["name"]:
            return Mark.RED
#3. Имеют одинаковые названия
        else:
            gas_sort1 = find_meta_value(gas_name1,'Сорт')
            gas_sort2 = find_meta_value(gas_name2,'Сорт')

            gas_mark1 = find_meta_value( gas_name1,'Марка')
            gas_mark2 = find_meta_value( gas_name2,'Марка')

            #3.1 Есть и сорт и марка
            if gas_sort1 is not None and gas_sort2 is not None and gas_mark1 is not None and gas_mark2 is not None: 
                #3.1.1
                if gas_sort1 == gas_sort2 and gas_mark1 == gas_mark2:
                    return Mark.GREEN
                #3.1.2
                else:
                    return Mark.ORANGE
            #3.2 Есть неполный набор о сорте и марке газов
            else:
                #3.2.1 
                #есть сорт газов
                if gas_sort1 is not None and gas_sort2 is not None:
                    if gas_sort1 == gas_sort2:
                        return Mark.GREEN
                    else:
                        return Mark.ORANGE
                
                #есть марки газа
                elif gas_mark1 is not None and gas_mark2 is not None:
                    if gas_mark1 == gas_mark2:
                        return Mark.GREEN
                    else:
                        return Mark.ORANGE 
                
                    #3.2.2.если данные в разных пармаетрах  
                elif  (gas_sort1 and gas_mark2 ) or  (gas_mark1 and gas_sort2):
                    return Mark.ORANGE
              #3.3
                elif (gas_sort1 and gas_mark1) is not None != (gas_sort2 and gas_mark2) is not None:
                    if gas_sort1 == gas_sort2 or gas_mark1 == gas_mark2:
                        return Mark.GREEN
                    else:
                        return Mark.ORANGE
                #3.4 Если для одного из моногазов не указаны его сорт и марка, а для другого указаны, то пара отмечается как оранжевая.
                elif ((gas_sort1 and gas_mark1) is not None) != ((gas_sort2 and gas_mark2) is not None):
                    return Mark.ORANGE
                #3.5. Если ни для одного из моногазов не указаны их сорт и марка, то пара отмечается как зелёная.
                else:
                    return  Mark.GREEN
                
            
                    
            

In [20]:
def get_data_from_repo(path: str, token: str, start_target: str = '', json_type: str = "universal", compress: bool = False,
                       no_blob_data: bool = True):
    """
    Downloads a file from iacpass repository.
    :param start_target: path starting from which data will be returned
    :param name: name of the file to be saved
    :param url: base url to the repository
    :param path: Path to the file
    :param compress: ...
    :param no_blob_data: ...
    :return: None
    """

    params = {"path": path, "json-type": json_type, "compress": compress, "no-blob-data": no_blob_data, 'start-target-concept-path': start_target}

    headers = {"Authorization": f"Bearer {token}"}

    return requests.get("https://iacpaas.dvo.ru/api/data/export/user-item", params=params, headers=headers)


def get_without_download_from_repo(path: str, token: str, start_target: str = '', json_type: str = "universal",
                                   compress: bool = False,
                                   no_blob_data: bool = True):
    """
    Get a data from iacpass repository.
    :param start_target: path starting from which data will be returned
    :param name: name of the file to be saved
    :param url: base url to the repository
    :param path: Path to the file
    :param compress: ...
    :param no_blob_data: ...
    :return: None
    """
    r = get_data_from_repo(path, token, start_target, json_type, compress, no_blob_data)
    if r.status_code == 200:
        response_json = r.json()

        return parse_nested_json(response_json)
    else:
        raise Exception("Failed to get data")

In [21]:
def get_token(username: str, password: str) -> str:
    """
    Get token from iacpass
    :param username: username
    :param password: password
    :return: token
    """
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    payload = {"username": username, "password": password}
    r = requests.post("https://iacpaas.dvo.ru/api/signin", json=payload, headers=headers)
    if r.status_code != 200:
        raise Exception("Failed to get token: {}".format(r.text))
    return r.json()["accessToken"]

In [22]:
def split_path(input_string):
    """Разделяет строку пути на две части: последнюю часть пути перед $ и начальный путь."""
    parts = input_string.split('$')
    if len(parts) != 2:
        raise ValueError("Неверный формат строки пути")

    root_path_parts = parts[0].strip().split('/')
    last_root_part = root_path_parts[-1]  # берем последний элемент

    initial_path = parts[1].strip()[:-1]  # Убираем точку с запятой в конце
    initial_path = initial_path.replace(' / ', '/')  # Заменяем " / " на "/"

    result1 = f"Загрузки / {last_root_part}"
    result2 = initial_path

    return result1, result2

In [23]:
gas_mix1 =find_name_value(json_file,"Аргон-Гелий Ar-95% + He-5%")
print(gas_mix1)

{'id': 17940078395552, 'name': 'Аргон-Гелий Ar-95% + He-5%', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Газовая смесь', 'successors': [{'id': 17940078395556, 'name': 'Соотношение компонентов газовой смеси', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Соотношение компонентов газовой смеси', 'successors': [{'id': 17940078395560, 'name': 1, 'type': 'НЕТЕРМИНАЛ', 'meta': 'Компонент смеси', 'successors': [{'id': 17940078395416, 'name': 'Аргон газообразный первый сорт', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Газ', 'link': 'База технологических газов$/Моногазы/Газы на основе Аргона/Аргон газообразный первый сорт;'}, {'id': 17940078395564, 'name': 'Концентрация', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Концентрация', 'successors': [{'id': 17940078395734, 'value': 95.0, 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'REAL', 'meta': 'Числовое значение'}, {'id': 17940078395738, 'value': '%', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': '%'}]}]}, {'id': 17940078395740, 'name': 2, 'type': 'НЕТЕРМИНАЛ', 'meta': 'Компонент смеси', 'suc

In [24]:
def parse_nested_json(data):
    """
    Recursively parse nested JSON strings into Python dictionaries.
    """
    if isinstance(data, dict):
        return {key: parse_nested_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [parse_nested_json(item) for item in data]
    elif isinstance(data, str):
        try:
            return parse_nested_json(json.loads(data))
        except (json.JSONDecodeError, TypeError):
            return data
    else:
        return data

In [42]:
def gasmix_compare(json1,json2):
    ##Посчитать сколько газов в газовой смеси
    gasmix_el_comp_1 = find_meta_value(json1,"Соотношение компонентов газовой смеси")
    elements_names_1 = [item["successors"][0]["name"] for item in gasmix_el_comp_1['successors']]
    
    ##масив для результатов сравнений
    color_results = []

    
    gasmix_el_comp_2 = find_meta_value(json2,"Соотношение компонентов газовой смеси")
    elements_names_2 = [item["successors"][0]["name"] for item in gasmix_el_comp_2['successors']]
    
    #1.Для каждого газа из смеси1 должен найтись такой же в смеси2
    
    if set(elements_names_1) != set(elements_names_2):
        return Mark.RED

    #3.Процентное содержание (объемная доля) соответствующих моногазов
    for element in gasmix_el_comp_1["successors"]:
        for element2 in gasmix_el_comp_2["successors"]:
            gas_1 = find_meta_value(element,"Газ")   
            gas_name_1 = gas_1["name"]   
            print("gas_name_1:" , gas_name_1)
            gas_2 = find_name_value(element2,gas_name_1)
            if gas_2 is None:
                continue
            gas_name_2 = gas_2["name"]
            

            if gas_name_1 == gas_name_2:
                concentrat_1 = find_meta_value(element,"Концентрация")
                concentrat_1 = concentrat_1["successors"][0]["value"]
               
                concentrat_2 = find_meta_value(element2,"Концентрация")
                concentrat_2 = concentrat_2["successors"][0]["value"]

                if abs(concentrat_1 - concentrat_2) < 10:
                  color_results.append(Mark.GREEN)
                elif abs(concentrat_1 - concentrat_2) < 30:
                  color_results.append(Mark.ORANGE)
                else:
                  color_results.append(Mark.RED)


              
    
    


   
     
    c = Counter(color_results)
    
    return c.most_common(1)[0][0]

In [43]:
print(gasmix_compare(gas_mix1,gas_mix1))



gas_name_1: Аргон газообразный первый сорт
gas_name_1: Аргон газообразный первый сорт
gas_name_1: Гелий газообразный марка А
gas_name_1: Гелий газообразный марка А
Mark.GREEN
